# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 1:13PM,243891,19,ADV80004824,"AdvaGen Pharma, Ltd",Released
1,Aug 10 2022 1:13PM,243891,19,ADV80004825,"AdvaGen Pharma, Ltd",Released
2,Aug 10 2022 1:13PM,243891,19,ADV80004826,"AdvaGen Pharma, Ltd",Released
3,Aug 10 2022 1:10PM,243890,10,8346378,"Snap Medical Industries, LLC",Released
4,Aug 10 2022 1:10PM,243890,10,8346379,"Snap Medical Industries, LLC",Released
5,Aug 10 2022 1:10PM,243890,10,8346381,"Snap Medical Industries, LLC",Released
6,Aug 10 2022 12:57PM,243888,19,60016519,"GUSA Granules USA, Inc.",Released
7,Aug 10 2022 12:55PM,243887,10,DNMD-23521-BO,Emerginnova,Released
8,Aug 10 2022 12:55PM,243887,10,DNMD-23537-BO,Emerginnova,Released
9,Aug 10 2022 12:55PM,243887,10,DNMD-23589-BO,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,243883,Released,5
27,243887,Released,7
28,243888,Released,1
29,243890,Released,3
30,243891,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243883,NaN,NaN,5.0
243887,NaN,NaN,7.0
243888,NaN,NaN,1.0
243890,NaN,NaN,3.0
243891,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243805,0.0,11.0,4.0
243808,32.0,16.0,7.0
243811,0.0,0.0,3.0
243828,0.0,1.0,0.0
243832,0.0,12.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243805,0,11,4
243808,32,16,7
243811,0,0,3
243828,0,1,0
243832,0,12,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243805,0,11,4
1,243808,32,16,7
2,243811,0,0,3
3,243828,0,1,0
4,243832,0,12,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243805,,11,4
1,243808,32,16,7
2,243811,,,3
3,243828,,1,
4,243832,,12,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 1:13PM,243891,19,"AdvaGen Pharma, Ltd"
3,Aug 10 2022 1:10PM,243890,10,"Snap Medical Industries, LLC"
6,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc."
7,Aug 10 2022 12:55PM,243887,10,Emerginnova
14,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation
19,Aug 10 2022 12:20PM,243881,19,ACG North America LLC
20,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation
21,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc."
23,Aug 10 2022 12:19PM,243879,19,ACG North America LLC
24,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 1:13PM,243891,19,"AdvaGen Pharma, Ltd",,,3
1,Aug 10 2022 1:10PM,243890,10,"Snap Medical Industries, LLC",,,3
2,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",,,1
3,Aug 10 2022 12:55PM,243887,10,Emerginnova,,,7
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,,,5
5,Aug 10 2022 12:20PM,243881,19,ACG North America LLC,,,1
6,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation,,,1
7,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc.",,1,1
8,Aug 10 2022 12:19PM,243879,19,ACG North America LLC,,,1
9,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc.",,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 1:13PM,243891,19,"AdvaGen Pharma, Ltd",3,,
1,Aug 10 2022 1:10PM,243890,10,"Snap Medical Industries, LLC",3,,
2,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",1,,
3,Aug 10 2022 12:55PM,243887,10,Emerginnova,7,,
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5,,
5,Aug 10 2022 12:20PM,243881,19,ACG North America LLC,1,,
6,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation,1,,
7,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc.",1,1,
8,Aug 10 2022 12:19PM,243879,19,ACG North America LLC,1,,
9,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc.",14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 1:13PM,243891,19,"AdvaGen Pharma, Ltd",3,,
1,Aug 10 2022 1:10PM,243890,10,"Snap Medical Industries, LLC",3,,
2,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",1,,
3,Aug 10 2022 12:55PM,243887,10,Emerginnova,7,,
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 1:13PM,243891,19,"AdvaGen Pharma, Ltd",3.0,NaN,NaN
1,Aug 10 2022 1:10PM,243890,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
2,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Aug 10 2022 12:55PM,243887,10,Emerginnova,7.0,NaN,NaN
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 1:13PM,243891,19,"AdvaGen Pharma, Ltd",3.0,0.0,0.0
1,Aug 10 2022 1:10PM,243890,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
2,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Aug 10 2022 12:55PM,243887,10,Emerginnova,7.0,0.0,0.0
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1707065,22.0,20.0,5.0
15,731587,35.0,9.0,0.0
16,243865,1.0,0.0,0.0
19,1463296,7.0,2.0,0.0
20,1219163,45.0,39.0,32.0
21,243828,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1707065,22.0,20.0,5.0
1,15,731587,35.0,9.0,0.0
2,16,243865,1.0,0.0,0.0
3,19,1463296,7.0,2.0,0.0
4,20,1219163,45.0,39.0,32.0
5,21,243828,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,22.0,20.0,5.0
1,15,35.0,9.0,0.0
2,16,1.0,0.0,0.0
3,19,7.0,2.0,0.0
4,20,45.0,39.0,32.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,22.0
1,15,Released,35.0
2,16,Released,1.0
3,19,Released,7.0
4,20,Released,45.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,5.0,0.0,0.0,0.0,32.0,0.0
Executing,20.0,9.0,0.0,2.0,39.0,1.0
Released,22.0,35.0,1.0,7.0,45.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,32.0,0.0
1,Executing,20.0,9.0,0.0,2.0,39.0,1.0
2,Released,22.0,35.0,1.0,7.0,45.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,32.0,0.0
1,Executing,20.0,9.0,0.0,2.0,39.0,1.0
2,Released,22.0,35.0,1.0,7.0,45.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()